In [1]:
import sys
import os

from typing import Optional, Union, Tuple
import numpy as np
from tqdm.notebook import tqdm

# add parent dir to be able to import utils file
parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import duqling_py.functions as functions
from duqling_interface import DuqlingInterface
duq_r = DuqlingInterface()

In [2]:
def lhs_array(n, d, seed=None) -> np.array:
    if seed is not None: np.random.seed(seed)
    samples = (np.arange(n)[:, None] + np.random.rand(n, d)) / n
    for i in range(d):
        samples[:, i] = np.random.permutation(samples[:, i])
    return samples

In [66]:
NUM_SAMPLES = 100
all_fnames = duq_r.list_functions().fname
failed_funcs = []

stochastic_piston_kwargs = dict( 
    Ta_generate = lambda:0.5, 
    P0_generate = lambda:0.5 
)

for fname in tqdm(all_fnames):

    if fname == 'dts_sirs': # currently broken
        continue

    func_info = duq_r.get_function_info(fname)
    input_dim = int(func_info['input_dim'][0])
    X = lhs_array(NUM_SAMPLES, input_dim)
    X_before = X.copy()
    
    duq_py_func = getattr(functions, fname)
    kwargs = stochastic_piston_kwargs if fname == 'stochastic_piston' else dict()
    y_p = np.apply_along_axis(duq_py_func, 1, X, **kwargs)
    X, y_r = duq_r.generate_data(fname, X=X, **kwargs)

    assert (X_before == X).all() # make sure X isn't mutated

    try:
        assert np.isclose(y_r, y_p).all()  # output array shapes should match and values should be more or less identical
    except:
        failed_funcs.append(fname)

  0%|          | 0/61 [00:00<?, ?it/s]

In [69]:
print("The following functions failed the above tests:")
for f in failed_funcs:
    print('\t- '+f)

The following functions failed the above tests:
	- simple_machine
	- vinet
	- ocean_circ
	- pollutant
	- beam_deflection
	- simple_machine_cm


In [85]:
incorrect_tranpose = []
for fname in failed_funcs:
    func_info = duq_r.get_function_info(fname)
    input_dim = int(func_info['input_dim'][0])
    X = lhs_array(NUM_SAMPLES, input_dim)
    X_after, y_r = duq_r.generate_data(fname, 1, X.copy())

    duq_py_func = getattr(functions, fname)
    y_r = duq_r.generate_data(fname, X=X)[1]
    y_p = np.array(list(map(duq_py_func, X)))

    if np.isclose(y_r, y_p.transpose()).all():
        incorrect_tranpose.append(fname)

print("Functions whose Python/R implementations are significantly different")
for func in set(failed_funcs) ^ set(incorrect_tranpose):
    print(f'\t- {func}')

print("\nFunctions that output inccorectly transposed arrays:")
for func in incorrect_tranpose:
    print(f'\t- {func}')

Functions whose Python/R implementations are significantly different
	- ocean_circ

Functions that output inccorectly transposed arrays:
	- simple_machine
	- vinet
	- pollutant
	- beam_deflection
	- simple_machine_cm


## Summary of issues

- $\texttt{ignition}$ uses different equation from paper
    - The paper uses $r^5 (1 + 100000 \times (1 + \operatorname{erf}(10 \times (r − 2))))$ 
    but the code uses $r^5 (1+100000 \times (2 \times \operatorname{cdf}(\sqrt{2}*10*(r-2))))$
- $\texttt{dts\_sirs}$ fails to terminate early when N[t] is null ('N[t] <= 0' breaks in this case)
- $\texttt{simple\_machine, vinet, pollutant, beam\_deflection, simple\_machine\_cm}$ transposed incorrectly
    - Currently, R duqling outputs arrays with shape (output_dim, n_samples) instead of the traditional (n_samples, output_dim):
    $$
    \begin{bmatrix}
        | & & | \\
        \text{sample 1} & \cdots & \text{sample n}\\
        | & & |
    \end{bmatrix}
    \quad
    \text{instead of}
    \quad
    \begin{bmatrix}
        & – & \text{sample 1} & – & \\
        & & \vdots & & \\
        & – & \text{sample n} & – &
    \end{bmatrix}
    $$
    - Is this the intended behavior?